In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline
pd.get_option("display.max_columns")

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.datasets import make_blobs
from scipy.stats import norm
cmap = sns.diverging_palette(220, 10, as_cmap=True) # цветовая палитра для seaborn 

import warnings
warnings.filterwarnings('ignore')

num_iterations = 10 # количество итераций для замера времени
accuracy = pd.DataFrame(columns=['accuracy'])

N, M, R_min, R_max = количество классов(4), количество элементов в классе(745), диапазон значений точности(мин = .700, мах = .750)
параметры, дающие значение accuraccy = 0.723: 
- n_samples = 766 (количество элементов всего, 766*6);
- centers = 6 (количество классов);
- n_features=2 (количество признаков);
- cluster_std = 1.85 (Стандартное отклонение кластеров);
- random_state=0 (для фиксации).


In [5]:
N, M, R_min, R_max = 4, 745*4, .700, .750

In [6]:
# Генерация четырех классов, каждый из которых представим в виде двух признаков
 
X, y = make_blobs(n_samples=M, centers=N, n_features=2,
                  cluster_std = 1.85,
                   random_state=0)
 
# first_feature, second_feature - признаки; target_value - метка класса
columns = {'first_feature': X[:, 0], 'second_feature': X[:, 1], 'target_value': y}
data = pd.DataFrame(data = columns)
data.head(10)

,first_feature,second_feature,target_value
0,3.228883,3.911824,1
1,5.563205,-0.581941,1
2,3.064169,5.221389,0
3,0.303633,5.844512,3
4,0.783235,2.475984,0
5,-1.583040,8.434516,3
6,-3.166997,5.046561,0
7,1.756726,4.217394,1
8,-0.462010,6.393290,3
9,0.974694,1.517609,1


Train-Test разбиение

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['target_value']), data['target_value'], test_size = 0.5, random_state = 42)

Реализация байесовского классификатора

In [8]:
class NaiveBayesClassifier:

  def fit(self, X_train, y_train):
    self.means = X_train.groupby(y_train).mean()
    self.stds = X_train.groupby(y_train).std()
    # априорная вероятность классов (количество элементов класса, деленное на общее количество элементов в тренировочном наборе)
    self.probs = X_train.groupby(y_train).apply(lambda x: len(x) / X_train.shape[0])
  
  def predict(self, X_test):
    y_pred = []
    for elem in range(X_test.shape[0]):
      p = {}
      for cl in np.unique(y_train):
        # априорная вероятность для каждого класса
        p[cl] = self.probs.iloc[cl]
        for index, param in enumerate(X_test.iloc[elem]):
          # вычисляется функция правдоподобия и сразу же умножается на полученную выше априорную вероятность класса
          #  P(H|A) ~ P(H)P(A|H), апостериорная вероятность ~ априорная вероятность * функцию правдопободия
          p[cl] *= norm.pdf(param, self.means.iloc[cl, index], self.stds.iloc[cl, index])
      y_pred.append(pd.Series(p).values.argmax())
    return y_pred

In [9]:
time_NB = []
for iter in range(num_iterations):
  start = time.time()
  model = NaiveBayesClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  time_NB.append(float(time.time() - start))


In [10]:
score = accuracy_score(y_test, y_pred)
accuracy.at['реализованный НБ', 'accuracy'] = score 

Оценка работы классификатора, матрица ошибок

In [11]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[222  54  49  36]
 [ 52 283  35   0]
 [ 71  42 243  32]
 [ 44   0  25 302]]


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.61      0.59       361
           1       0.75      0.76      0.76       370
           2       0.69      0.63      0.66       388
           3       0.82      0.81      0.82       371

    accuracy                           0.70      1490
   macro avg       0.71      0.71      0.70      1490
weighted avg       0.71      0.70      0.71      1490

